# Introduction
In notebook 42 and 45 I made sure that every Fatty acid chain needed was produced in our model. Now, as we will apply the restrictive approach to our model, I will combine them all into a generic acyl-chain that reflects the distribution identified i Tang et al. (2009). The generic acyl chain will then need to have a chemical formula that matches the distribution given so that we maintain mass balance.

Finally, this chain will be attached to the phospholipid heads, and these metabolites modified so that mass balance is achieved.

Finally, we will add the fatty acid requirement to biomass.

In [1]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [2]:
model_e_coli = cameo.load_model('iML1515')

In [8]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [4]:
#add general Acyl-ACP reaction
model.add_metabolites(Metabolite(id='acylACP_c', name = 'Generic Acyl-ACP fatty acid', compartment = 'c', charge = -1, formula = 'C27.07H51.14O8N2PRS'))

In [5]:
model.metabolites.acylACP_c.annotation['sbo'] = 'SBO:0000247'

Note: the chemical formula of the generic acyl ACP is calculated by considering the chemical formula of each individual FA, and weighing it with the stoichiometry that the data provides.

In [6]:
#add reaction forming generic Acyl-ACP
model.add_reaction(Reaction(id='ACYLACP', name = 'Pseudoreaction to create general acyl-ACP chain'))

In [7]:
model.reactions.ACYLACP.annotation['sbo'] = 'SBO:0000631'

In [8]:
model.reactions.ACYLACP.add_metabolites({
    model.metabolites.get_by_id('2mtridecACP_c'):-0.004,
    model.metabolites.myrsACP_c:-0.025,
    model.metabolites.get_by_id('3mtdecACP_c'):-0.096,
    model.metabolites.get_by_id('2mtdecACP_c'):-0.006,
    model.metabolites.get_by_id('pdecACP_c'):-0.027,
    model.metabolites.get_by_id('2mpdecACP_c'): -0.279,
    model.metabolites.get_by_id('palmACP_c'): -0.347,
    model.metabolites.get_by_id('3mhdecACP_c'): -0.114,
    model.metabolites.get_by_id('2mhdecACP_c'): -0.037,
    model.metabolites.hdecACP_c:-0.024,
    model.metabolites.ocdcaACP_c:-0.041,
    model.metabolites.acylACP_c:1
})

In [4]:
model.reactions.ACYLACP.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

27.07 is not an integer (in formula C27.07H51.14O8N2PRS)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

51.14 is not an integer (in formula C27.07H51.14O8N2PRS)



{'C': -3.552713678800501e-15}

Note, here we don't have complete mass balance because to define the generic acyl-ACP (and acyl-CoA) we need to have decimals in the chemical formula. But, as the imbalance here is sooo small it is oke and we will continue on with it.

In [10]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [9]:
#convert the acylACP to acyl-CoA metabolite.
#add acyl-CoA metabolite first
model.add_metabolites(Metabolite(id='acylcoa_c', name = 'Generic Acyl-CoA', compartment = 'c', charge = -3, formula = 'C37.07H63.14O17N7P3S'))

In [10]:
model.metabolites.acylcoa_c.annotation['sbo'] = 'SBO:0000247'

In [11]:
#add reaction to convert the two
model.add_reaction(Reaction(id='ACYAC', name= 'CoA:[acyl-carrier-protein] transferase'))

In [12]:
model.reactions.ACYAC.bounds = (-1000,1000)

In [13]:
model.reactions.ACYAC.add_metabolites({
    model.metabolites.ACP_c:-1,
    model.metabolites.acylcoa_c:-1,
    model.metabolites.coa_c:1,
    model.metabolites.acylACP_c:1,
    model.metabolites.h_c:1
})

In [14]:
model.reactions.ACYAC.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

37.07 is not an integer (in formula C37.07H63.14O17N7P3S)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

63.14 is not an integer (in formula C37.07H63.14O17N7P3S)



{}

In [15]:
#remove acoa_c metabolite, a generic acyl-coa that was present in the model.
model.remove_metabolites(model.metabolites.acoa_c)

In [16]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

When we have the generic acyl-coa molecule made, we can the incorporate it into the glycerolipid heads and ensure the three lipids we need for the biomass reaction can be made. 

The acyl-Coa chains are incorporated into the sn-glycerol-3-phosphate molecule, in two steps to give a 1,2-diacyl-glycerol-3-phosphate. this can then react further into the three glycerolipids we need: Phosphatidylglycerol, Diphosphatidylglycerol and Phosphatidylethanolamine. 

To incorporate the chains, we just need to change the chemical formula of certain metabolites and change some reactions to accomodate the generic acyl-chains.

In [17]:
#start in reaction ACCOAACT: change the acoa to the generic acylcoa_c met I made
model.reactions.ACCOAACT.add_metabolites({
    model.metabolites.acylcoa_c:-1
})

In [18]:
#change formula of aglyc3p_c
model.metabolites.aglyc3p_c.formula = 'C19.07H38.14O7P'

In [19]:
#check mass balance
model.reactions.ACCOAACT.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

19.07 is not an integer (in formula C19.07H38.14O7P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C19.07H38.14O7P)



{}

In [20]:
#do the same for the ACCOATT reaction
model.reactions.ACCOATT.add_metabolites({
    model.metabolites.acylcoa_c:-1
})

In [21]:
#change formula of pa_c
model.metabolites.pa_c.formula = 'C35.14H69.28O8P'

In [22]:
#check mass balance
model.reactions.ACCOATT.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

35.14 is not an integer (in formula C35.14H69.28O8P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

69.28 is not an integer (in formula C35.14H69.28O8P)



{}

In [23]:
#then change the cdpdag_c  chemical formula accordingly
model.metabolites.cdpdag_c.formula = 'C44.14H79.28N3O15P2'

In [24]:
model.reactions.CTPPCT.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

44.14 is not an integer (in formula C44.14H79.28N3O15P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

79.28 is not an integer (in formula C44.14H79.28N3O15P2)



{}

In [25]:
#onto phosphatidyl glycerol, via pgp_c
model.metabolites.pgp_c.formula = 'C38.14H76.28O13P2'

In [26]:
model.reactions.CDPPT.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C38.14H76.28O13P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

76.28 is not an integer (in formula C38.14H76.28O13P2)



{}

In [27]:
#then onto the pg_c (phosphatidyl glycerol)
model.metabolites.pg_c.formula = 'C38.14H75.28O10P'

In [28]:
model.reactions.PGP.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C38.14H75.28O10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

75.28 is not an integer (in formula C38.14H75.28O10P)



{}

In [29]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [30]:
#now onto cardiolipin
model.metabolites.clpn_c.formula = 'C73.28H142.56O17P2'

In [31]:
model.reactions.CLPNS.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

73.28 is not an integer (in formula C73.28H142.56O17P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

142.56 is not an integer (in formula C73.28H142.56O17P2)



{}

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [87]:
#then now make sure phosphatidylethanolamin can be made properly, through ps_c
model.metabolites.ps_c.formula = 'C38.14H74.28NO10P'

In [88]:
model.reactions.CDPDAGPT.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

44.14 is not an integer (in formula C44.14H79.28N3O15P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

79.28 is not an integer (in formula C44.14H79.28N3O15P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C38.14H74.28NO10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

74.28 is not an integer (in formula C38.14H74.28NO10P)



{}

In [91]:
model.metabolites.pe_c.formula = 'C37.14H74.28NO8P'

In [92]:
model.reactions.PSD.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C38.14H74.28NO10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

74.28 is not an integer (in formula C38.14H74.28NO10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

37.14 is not an integer (in formula C37.14H74.28NO8P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

74.28 is not an integer (in formula C37.14H74.28NO8P)



{}

In [93]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now that we have made all the generic lipids that we need, we can go ahead and add them to our biomass reaction. We know we have 9% (in g/g) of lipids in our biomass, and we know the distribution of the lipids as 25/50/25 (in g/g) for pg, clpn and pe respectively. With this we can calculate the mmol/gCDW to add as a stoichiometry to our model.

This leads to the following stoichiometries:
- pg_c: 0.031037
- clpn_c: 0.033142
- pe_c: 0.032424

In [104]:
#add these metabolites to biomass
model.reactions.biomass.add_metabolites({
    model.metabolites.pg_c:-0.031037,
    model.metabolites.clpn_c:-0.033142,
    model.metabolites.pe_c:-0.032424
})

Check to see the effect on the growth. You can see the growth rate decreases a bit to 0.700, but this is oke and would be somewhat expected. 

In [105]:
model.optimize()

,fluxes,reduced_costs
IDPh,1.727336e+00,0.000000e+00
CAT,1.829815e-02,0.000000e+00
PDHam1hi,0.000000e+00,5.551115e-17
HYDA,0.000000e+00,-0.000000e+00
MALHYDRO,-6.938894e-18,0.000000e+00
...,...,...
3HMAD160,5.066520e-02,0.000000e+00
EMAR160x,0.000000e+00,-7.378930e-03
EMAR160y,5.066520e-02,-3.365364e-16
ACYLACP,1.815957e-01,-1.799776e-17


In [106]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finally, I want to check what the total biomass reaction adds up to. I.e. all metabolites together should not add up to more than 1 g/gcdw. 
Note: we still need to add bacillithiol to this, but this will not be a huge effect.

In [107]:
#check total g/gdcw biomass
biomass_g_g = []
for met in model.reactions.biomass.metabolites:
    stoich = model.reactions.biomass.metabolites.get(met) #get stoichiometry in mmol/gcdw
    if stoich == 1 :#ignore the biomass part, but need to include the ADP/Pi/UDP that is also made here
        continue
    else:
        mw = met.formula_weight # get molecular weight of metabolite in g/mol
        try:
            weight_fraction = (mw/1000)*stoich #the g/gcdw of this metabolite
            biomass_g_g.append(weight_fraction)
        except TypeError: #for the met with R in it, just ignore
            print(met.id)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

38.14 is not an integer (in formula C38.14H75.28O10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

75.28 is not an integer (in formula C38.14H75.28O10P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

73.28 is not an integer (in formula C73.28H142.56O17P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

142.56 is not an integer (in formula C73.28H142.56O17P2)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

37.14 is not an integer (in formula C37.14H74.28NO8P)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\co

In [108]:
#to find total g/gcdw of all mets, sum up this list
abs(sum(biomass_g_g))

1.117769768901451